In [ ]:
#scrapowanie brakujących książek

In [ ]:
#!py -m pip install selenium
#!py -m pip install webdriver_manager

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [36]:
 
def get_book_titles_authors_links(driver, url):
    books = []
    try:
        driver.get(url)
        time.sleep(2)  # Poczekaj na załadowanie strony
        
        # Pobranie wszystkich kontenerów książek
        book_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'bookCard_card__Cfo9J')]")
        
        for container in book_containers:
            try:
                # Pobranie tytułu
                title_element = container.find_element(By.XPATH, ".//div[contains(@class, 'bookCard_footerText__Mjunj')]/p[1]")
                title = title_element.text if title_element else "Brak tytułu"
                
                # Pobranie autora
                author_element = container.find_element(By.XPATH, ".//div[contains(@class, 'bookCard_footerText__Mjunj')]/p[2]")
                author = author_element.text if author_element else "Brak autora"
                
                # Pobranie linku
                link_element = container.find_element(By.XPATH, ".//a")
                href = link_element.get_attribute("href") if link_element else "Brak linku"
                
                # Dodanie wyników do listy
                books.append({"Title": title, "Author": author, "URL": href})
            except Exception as e_inner:
                print(f"Error processing book container: {e_inner}")
    except Exception as e:
        print(f"Error scraping page: {e}")
    return books

# Przykład użycia:
# driver = webdriver.Chrome()
# books_data = get_book_titles_authors_links(driver, "URL_DO_STRONY")
# print(books_data)

In [38]:
import pandas as pd
# Konfiguracja Selenium
driver = webdriver.Chrome()  # Możesz użyć odpowiedniego sterownika np. dla Firefoxa: webdriver.Firefox()

# Podstawowy URL
base_url = "https://www.bookbeat.com/pl/search?page={}&query=Polski&Category=148&format=audiobook&language=polish"

# Dane do porównania
all_titles_authors_links = []

for page in range(1, 201):  # Zakres stron
    page_url = base_url.format(page)
    books_on_page = get_book_titles_authors_links(driver, page_url)
    all_titles_authors_links.extend(books_on_page)

# Tworzenie ramki danych z listy książek
df_titles_authors_links = pd.DataFrame(all_titles_authors_links)
df_titles_authors_links.to_csv("all_titles_authors_links.csv", index=False, encoding="utf-8")


In [39]:
import pandas as pd

# Wczytanie danych z plików CSV
# Dodaj nazwy kolumn do ramki bookBeat_all przy wczytywaniu
bookbeat_all = pd.read_csv("bookBeat_data4.csv")
scraped_data = pd.read_csv("all_titles_authors_links.csv")

# Usunięcie duplikatów z ramki `bookbeat_all`
bookbeat_all_cleaned = bookbeat_all.drop_duplicates()

# Porównanie kolumn "Title" i "Author" między ramkami
bookbeat_titles_authors = bookbeat_all_cleaned[["Title", "Author"]]
scraped_titles_authors = scraped_data[["Title", "Author"]]

# Łączenie danych w celu znalezienia unikalnych książek w scraped_data
merged = pd.merge(scraped_data, bookbeat_titles_authors, on=["Title", "Author"], how="left", indicator=True)
new_books = merged[merged["_merge"] == "left_only"].drop(columns=["_merge"])

# Zapisanie wyników do pliku CSV
new_books.to_csv("new_books.csv", index=False, encoding="utf-8")

# Wyświetlenie podsumowania
print(f"Znaleziono {len(new_books)} nowych książek, które nie są obecne w bookBeat_data4.csv.")


Znaleziono 1666 nowych książek, które nie są obecne w bookBeat_data4.csv.


In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Funkcja do scrapowania danych o książce
def scrape_book_details(driver, url, xpaths):
    results = {}
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 1.5)

        # Akceptowanie ciasteczek
        try:
            accept_cookies_button = wait.until(
                EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", accept_cookies_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", accept_cookies_button)
        except Exception:
            pass

        # Klikanie "Pokaż więcej" (jeśli dostępne)
        try:
            show_more_button = wait.until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[1]/div/button'))
            )
            if show_more_button.is_displayed() and show_more_button.is_enabled():
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_more_button)
                time.sleep(1)
                show_more_button.click()
        except Exception:
            pass

        # Scrapowanie elementów
        for name, xpath in xpaths.items():
            try:
                if name == "Category":
                    elements = driver.find_elements(By.XPATH, xpath)
                    categories = [el.text for el in elements]
                    results[name] = ", ".join(categories) if categories else "NA"
                else:
                    element = driver.find_element(By.XPATH, xpath)
                    results[name] = element.text
            except Exception:
                results[name] = "NA"

            # Autor
            try:
                single_author_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/a[1]/div/div/span"
                multiple_authors_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/div[1]/div[2]/div/a/span"
                elements = driver.find_elements(By.XPATH, multiple_authors_xpath)
                if elements:
                    results["Author"] = ", ".join([el.text for el in elements])  # Łączy autorów w jeden ciąg
                else:
                    results["Author"] = driver.find_element(By.XPATH, single_author_xpath).text  # Pojedynczy autor
            except Exception:
                results["Author"] = "NA"

            # Lektor
            try:
                single_lector_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/a[2]/div/div/span"
                multiple_lectors_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/div[2]/div[2]/div/a/span"
                elements = driver.find_elements(By.XPATH, multiple_lectors_xpath)
                if elements:
                    results["Lector"] = ", ".join([el.text for el in elements])  # Łączy lektorów w jeden ciąg
                else:
                    results["Lector"] = driver.find_element(By.XPATH, single_lector_xpath).text  # Pojedynczy lektor
            except Exception:
                results["Lector"] = "NA"

        # Scrapowanie daty publikacji
        try:
            publication_date = wait.until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[1]/div/div/div/div[1]/p[2]'))
            ).text
            results["Publication_Date"] = publication_date
        except Exception:
            results["Publication_Date"] = "NA"

        # Scrapowanie informacji o tomie (serii)
        try:
            volume_info = driver.find_element(
                By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/a[3]/div/div/span'
            ).text
            results["Volume_Info"] = volume_info
        except Exception:
            results["Volume_Info"] = "Nie dotyczy"

    except Exception as e:
        print(f"Error scraping book details: {e}")
    return results


In [41]:
# Wczytanie linków książek z pliku CSV
new_books = pd.read_csv("new_books.csv")

In [42]:
# Funkcja do scrapowania danych dla książek z `new_books`
def scrape_new_books_data(driver, books_df):
    all_books_data = []
    
    # Definiowanie XPath-ów
    xpaths = {
        "Title": "/html/body/div[1]/div[1]/div[2]/main/section[2]/div/div/div[1]/h1",
        "Book_Score": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/button/div/figure[1]/div/figcaption",
        "Number_of_ratings_book": "/html/body/div[1]/div[1]/div[2]/main/section[4]/div/div/div/div/div[2]/figure[1]/p",
        "Lector_Score": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/button/div/figure[2]/div/figcaption",
        "Number_of_ratings_lector": "/html/body/div[1]/div[1]/div[2]/main/section[4]/div/div/div/div/div[2]/figure[2]/p",
        "Duration": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/span/div/span",
        "Category": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[3]/div[1]/div/div[1]/div/a",
        "Format": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[3]/div[1]/div/div[2]/div/a",
        "Language": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[3]/div[1]/div/div[3]/p[2]",
    }

    for index, row in books_df.iterrows():
        book_url = row['URL']
        print(f"Scraping data for book: {book_url}")
        book_data = scrape_book_details(driver, book_url, xpaths)
        all_books_data.append(book_data)
    
    return all_books_data


In [43]:
# Tworzenie przeglądarki tylko raz 
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Scrapowanie danych dla nowych książek
    new_books_data = scrape_new_books_data(driver, new_books)
    
    # Zapisanie wyników do pliku CSV
    new_books_df = pd.DataFrame(new_books_data)
    new_books_df.to_csv("scraped_new_books_data.csv", index=False, encoding="utf-8")
    print("Scraping completed and data saved to 'scraped_new_books_data.csv'.")

finally:
    driver.quit()

Scraping data for book: https://www.bookbeat.com/pl/book/grobowa-cisza-zalobny-zgielk-1528338
Scraping data for book: https://www.bookbeat.com/pl/book/bo-nie-zacznij-odmawiac-i-zyc-po-swojemu-1458346
Scraping data for book: https://www.bookbeat.com/pl/book/7-nawykow-skutecznego-dzialania-wydanie-jubileuszowe-542466
Scraping data for book: https://www.bookbeat.com/pl/book/dobry-material-1441217
Scraping data for book: https://www.bookbeat.com/pl/book/zasniezeni-1506877
Scraping data for book: https://www.bookbeat.com/pl/book/dawno-temu-w-warszawie-1172142
Scraping data for book: https://www.bookbeat.com/pl/book/disney-w-glowie-sie-nie-miesci-1465437
Scraping data for book: https://www.bookbeat.com/pl/book/one-1525818
Scraping data for book: https://www.bookbeat.com/pl/book/wrony-582125
Scraping data for book: https://www.bookbeat.com/pl/book/disney-vaiana-skarb-oceanu-1527409
Scraping data for book: https://www.bookbeat.com/pl/book/sherlock-holmes-najslynniejsze-sledztwa-1480593
Scrapin

In [45]:
import pandas as pd

# Wczytaj dane z plików CSV lub przekształć dane w ramki danych
# Zakładam, że masz dwie tabelki w formie DataFrame
# Jeśli dane są w plikach CSV, wczytaj je:
bookBeat_data = pd.read_csv("bookBeat_data4.csv")
scraped_new_books_data = pd.read_csv("scraped_new_books_data.csv")

# Połączenie danych
combined_data = pd.concat([bookBeat_data, scraped_new_books_data], ignore_index=True)

combined_data = combined_data.drop_duplicates()

# Zapisanie wyniku do pliku CSV
combined_data.to_csv("bookBeat_data5.csv", index=False)

print("Połączono dane. Wynik zapisano w 'bookBeat_data5.csv'.")


Połączono dane. Wynik zapisano w 'bookBeat_data5.csv'.


In [46]:
bookBeat_data=pd.read_csv("bookBeat_data5.csv")
bookBeat_data=bookBeat_data.drop_duplicates()
bookBeat_data.to_csv("bookBeat_dat6.csv", index=False)

In [48]:
import pandas as pd

# Załaduj dane
combined_data = pd.read_csv("bookBeat_dat6.csv")

# Sortowanie danych według liczby ocen w malejącej kolejności
# Zmień "Number_of_ratings_book" na nazwę kolumny odpowiadającej liczbie ocen w Twojej ramce danych
sorted_data = combined_data.sort_values(by="Number_of_ratings_book", ascending=False)

# Usunięcie duplikatów na podstawie wybranych kolumn
columns_to_check = ["Title", "Author", "Lector", "Publication_Date", "Volume_Info", "Duration", "Category"]
deduplicated_data = sorted_data.drop_duplicates(subset=columns_to_check, keep="first")

# Zapisanie wyniku do pliku CSV
deduplicated_data.to_csv("bookBeat_data7.csv", index=False)

print("Usunięto duplikaty z zachowaniem rekordów o największej liczbie ocen. Wynik zapisano w 'bookBeat_data7.csv'.")


Usunięto duplikaty z zachowaniem rekordów o największej liczbie ocen. Wynik zapisano w 'bookBeat_data7.csv'.


In [64]:
import pandas as pd
# Załaduj dane
combined_data = pd.read_csv("bookBeat_data7.csv")
#combined_data.head
combined_data.dtypes

Title                       object
Author                      object
Lector                      object
Book_Score                  object
Number_of_ratings_book      object
Lector_Score                object
Number_of_ratings_lector    object
Duration                    object
Category                    object
Format                      object
Language                    object
Publication_Date            object
Volume_Info                 object
dtype: object

In [70]:
#cleaning steps
import re
# Funkcja do czyszczenia wartości w kolumnach
def clean_score(value):
    if pd.isna(value) or value == "nan":  # Obsługa wartości NaN
        return None
    value = str(value)  # Konwersja na ciąg znaków, aby obsłużyć różne typy
    value = re.sub(r"0+", "", value)  # Usuń wszystkie zera
    value = value.replace(",", ".")  # Zamień przecinek na kropkę
    try:
        return float(value)  # Konwersja do typu float
    except ValueError:
        return None  # Jeśli konwersja się nie powiedzie, zwróć None

# Czyszczenie i konwersja kolumn do float
combined_data["Book_Score"] = combined_data["Book_Score"].apply(clean_score)
combined_data["Lector_Score"] = combined_data["Lector_Score"].astype("float")

# Sprawdzenie typów kolumn po konwersji
print(combined_data.dtypes)

# Sprawdzenie unikalnych wartości po czyszczeniu
print("Unikalne wartości w kolumnie Book_Score:", combined_data["Book_Score"].unique())
print("Unikalne wartości w kolumnie Lector_Score:", combined_data["Lector_Score"].unique())



Title                        object
Author                       object
Lector                       object
Book_Score                  float64
Number_of_ratings_book       object
Lector_Score                float64
Number_of_ratings_lector     object
Duration                     object
Category                     object
Format                       object
Language                     object
Publication_Date             object
Volume_Info                  object
dtype: object
Unikalne wartości w kolumnie Book_Score: [nan 4.7 3.9 4.5 4.4 3.6 4.3 4.6 2.7 4.2 4.8 4.1 3.3 4.9 3.8 4.  3.4 3.7
 3.5 3.1 5.  2.9 3.  2.5 2.8 3.2 2.1 2.2 2.6]
Unikalne wartości w kolumnie Lector_Score: [nan 4.2 4.5 4.4 4.6 4.3 4.8 4.7 4.1 4.9 2.4 5.  3.7 4.  3.6 3.8 3.9 3.5
 3.1 2.2 3.4 2.3 3.3 3.2 1.9 2.8 1.3 3.  2.6 2.7 2.5]


In [79]:
#cleaning steps
#Title, Author,Lector Book_Score, Number_of_ratings_book, Lector_Score, Number_of_ratings_lector, Duration, Category, Format, Language, Publication_Date, Volume_Info
#nan w ocenach książki i lektora na 0(ocen brak lub za mało do wystawienia, bo najn iższa ogólnie to 2 coś)
combined_data["Book_Score"] = combined_data["Book_Score"].fillna(0)
combined_data["Lector_Score"] = combined_data["Lector_Score"].fillna(0)
#usunąć slowo 'oceny' z Number_book_ratings i Nmbers_lector_ratings
combined_data["Number_of_ratings_book"] = combined_data["Number_of_ratings_book"].str.replace("oceny", "").str.strip()
combined_data["Number_of_ratings_lector"] = combined_data["Number_of_ratings_lector"].str.replace("oceny", "").str.strip()

# Sprawdzenie i zmiana wartości w "Number_of_ratings_book" na 0 tam, gdzie "Book_Score" wynosi 0
combined_data.loc[combined_data["Book_Score"] == 0, "Number_of_ratings_book"] = 0
combined_data.loc[combined_data["Lector_Score"] == 0, "Number_of_ratings_lector"] = 0

combined_data["Number_of_ratings_book"] = combined_data["Number_of_ratings_book"].fillna(1)
combined_data["Number_of_ratings_lector"] = combined_data["Number_of_ratings_lector"].fillna(1) #pozostałe nieznane wartości, które mają podany score

combined_data[["Number_of_ratings_book", "Number_of_ratings_lector"]] = combined_data[["Number_of_ratings_book", "Number_of_ratings_lector"]].astype("int")

In [78]:
# Zapisanie wyniku do pliku CSV
#combined_data.to_csv("bookBeat_data8.csv", index=False)

In [101]:
combined_data=pd.read_csv("bookBeat_data8.csv")

In [102]:
for column in ["Number_of_ratings_book", "Number_of_ratings_lector", "Book_Score", "Lector_Score"]:
    print(f"Unikalne wartości w kolumnie {column}: {combined_data[column].unique()}")


Unikalne wartości w kolumnie Number_of_ratings_book: [0.0000e+00 9.9500e+02 9.9000e+01 9.8600e+02 9.8100e+02 9.8000e+01
 9.7700e+02 9.7600e+02 9.7300e+02 9.7000e+01 9.6200e+02 9.6100e+02
 9.6000e+01 9.5700e+02 9.5560e+03 9.5500e+02 9.5400e+02 9.5200e+02
 9.5100e+02 9.5000e+01 9.4900e+02 9.4700e+02 9.4680e+03 9.4600e+02
 9.4300e+02 9.4000e+01 9.3800e+02 9.3600e+02 9.3000e+01 9.2500e+02
 9.2400e+02 9.2300e+02 9.2100e+02 9.2000e+02 9.2000e+01 9.1900e+02
 9.1800e+02 9.1700e+02 9.1600e+02 9.1400e+02 9.1200e+02 9.1100e+02
 9.1000e+02 9.1000e+01 9.0800e+02 9.0500e+02 9.0000e+01 8.9800e+02
 8.9100e+02 8.9000e+01 8.8800e+02 8.8700e+02 8.8600e+02 8.8500e+02
 8.8300e+02 8.8100e+02 8.8000e+01 8.7900e+02 8.7500e+02 8.7400e+02
 8.7000e+02 8.7000e+01 8.6800e+03 8.6600e+02 8.6000e+02 8.6000e+01
 8.5600e+02 8.5400e+02 8.5100e+02 8.5000e+02 8.5000e+01 8.4700e+02
 8.4400e+02 8.4100e+02 8.4000e+02 8.4000e+01 8.3900e+02 8.3800e+02
 8.3700e+02 8.3500e+02 8.3000e+02 8.3000e+01 8.2800e+02 8.2700e+02
 8.2400e+

In [103]:
#Title, Author,Lector Book_Score, Number_of_ratings_book, Lector_Score, Number_of_ratings_lector, Duration, Category, Format, Language, Publication_Date, Volume_Info
combined_data[["Title", "Author", "Lector", "Volume_Info"]] = combined_data[["Title", "Author", "Lector", "Volume_Info"]].astype("string")
combined_data["Publication_Date"] = pd.to_datetime(combined_data["Publication_Date"], dayfirst=True, errors='coerce')

In [104]:
#Category, Volume_Info

# Usunięcie wierszy, gdzie 'Category' to 'E-book'
combined_data = combined_data[combined_data["Category"] != "E-book"]

# Zamień kolumnę Publication_Date na daty (jeśli jeszcze tego nie zrobiłeś)
combined_data["Publication_Date"] = pd.to_datetime(combined_data["Publication_Date"], errors="coerce")
combined_data = combined_data[combined_data["Publication_Date"].dt.year <= 2024]

# Sprawdź wyniki
combined_data["Duration"].unique



<bound method Series.unique of 0        15 godz. 13 min.
1        17 godz. 19 min.
2         18 godz. 9 min.
3         7 godz. 28 min.
4         9 godz. 36 min.
               ...       
10152     7 godz. 49 min.
10153     6 godz. 58 min.
10154     7 godz. 41 min.
10155     6 godz. 39 min.
10156    11 godz. 22 min.
Name: Duration, Length: 10035, dtype: object>

In [105]:
#Duration, 
def convert_to_hours(duration):
    if pd.isna(duration):
        return None
    # Usunięcie jednostek "godz." i "min."
    duration = re.sub(r"godz\.|min\.", "", duration)
    # Rozdzielenie godzin i minut
    parts = duration.split()
    hours = float(parts[0])  # Godziny
    minutes = float(parts[1]) if len(parts) > 1 else 0  # Minuty
    # Obliczenie liczby godzin (minuty / 60)
    total_hours = hours + minutes/60
    return total_hours

# Przekształcenie kolumny 'Duration'
combined_data["Duration"] = combined_data["Duration"].apply(convert_to_hours)

# Sprawdzenie wyników
#combined_data["Duration"].head()


In [107]:
# Sprawdź wyniki
combined_data.head()


,Title,Author,Lector,Book_Score,Number_of_ratings_book,Lector_Score,Number_of_ratings_lector,Duration,Category,Format,Language,Publication_Date,Volume_Info
0,Omega,Marcin Szczygielski,Agnieszka Krzysztoń,0.0,0.0,0.0,0.0,15.216667,"Dla młodzieży, Dla młodzieży w wieku 12+",Audiobook,Polski,2021-11-30,Nie dotyczy
1,Laurowy pean,Julia Gambrot,Agnieszka Krzysztoń,0.0,0.0,4.2,10.0,17.316667,Powieści,Audiobook,Polski,2022-07-27,Nie dotyczy
2,Gniazda. Tom I. Naznaczeni przez Tyche,Piotr Walczak,Krzysztof Plewako-Szczerbiński,0.0,0.0,0.0,0.0,18.150000,Sci-fi,Audiobook,Polski,2023-02-06,Tom 1 - Gniazda
3,Pani K.,Marta Knopik,Cieślik,0.0,0.0,0.0,0.0,7.466667,Powieści,Audiobook,Polski,2022-10-05,Nie dotyczy
4,Wplątani,Arkadiusz Raczka,Roch Siemianowski,0.0,0.0,0.0,0.0,9.600000,"Powieści, Powieści historyczne, Klasyka",Audiobook,Polski,2022-09-22,Nie dotyczy


In [108]:
combined_data["Duration"] = combined_data["Duration"].round(2)

In [109]:
combined_data.head()

,Title,Author,Lector,Book_Score,Number_of_ratings_book,Lector_Score,Number_of_ratings_lector,Duration,Category,Format,Language,Publication_Date,Volume_Info
0,Omega,Marcin Szczygielski,Agnieszka Krzysztoń,0.0,0.0,0.0,0.0,15.22,"Dla młodzieży, Dla młodzieży w wieku 12+",Audiobook,Polski,2021-11-30,Nie dotyczy
1,Laurowy pean,Julia Gambrot,Agnieszka Krzysztoń,0.0,0.0,4.2,10.0,17.32,Powieści,Audiobook,Polski,2022-07-27,Nie dotyczy
2,Gniazda. Tom I. Naznaczeni przez Tyche,Piotr Walczak,Krzysztof Plewako-Szczerbiński,0.0,0.0,0.0,0.0,18.15,Sci-fi,Audiobook,Polski,2023-02-06,Tom 1 - Gniazda
3,Pani K.,Marta Knopik,Cieślik,0.0,0.0,0.0,0.0,7.47,Powieści,Audiobook,Polski,2022-10-05,Nie dotyczy
4,Wplątani,Arkadiusz Raczka,Roch Siemianowski,0.0,0.0,0.0,0.0,9.60,"Powieści, Powieści historyczne, Klasyka",Audiobook,Polski,2022-09-22,Nie dotyczy


In [110]:
# Zapisanie wyniku do pliku CSV
combined_data.to_csv("bookBeat_data9.csv", index=False)


In [112]:
combined_data=pd.read_csv("bookBeat_data9.csv")
import numpy as np
# Przeniesienie wartości typu "Tom X - ..." do Volume tylko wtedy, gdy pasują do wzorca
combined_data.loc[
    combined_data["Lector"].str.contains(r"^Tom \d+", na=False), "Volume"
] = combined_data["Lector"]

# Ustawienie NaN w kolumnie 'Lector' tylko dla pasujących rekordów
combined_data.loc[
    combined_data["Lector"].str.contains(r"^Tom \d+", na=False), "Lector"
] = np.nan


In [113]:
# Zapisanie wyniku do pliku CSV
combined_data.to_csv("bookBeat_data10.csv", index=False)

In [115]:
female_names =["Abigail", "Ada", "Adela", "Adelajda", "Adelina", "Adriana", "Adrianna", "Afra", "Afrodyta", "Afrodyzja", "Agata", "Agatka", "Aglaja", "Agniesia", "Agnieszka", "Agnisia", "Agrypina", "Aimée", "Alberta", "Albina", "Aldona", "Aleksandra", "Aleksja", "Alfreda", "Alicja", "Alina", "Alka", "Amalia", "Amanda", "Ambrozja", "Amelia", "Ana", "Anabella", "Anastazja", "Anatolia", "Ańcia", "Andrea", "Andromeda", "Andrzeja", "Andżelika", "Aneta", "Anetta", "Angela", "Angelika", "Ania", "Aniela", "Anita", "Anka", "Anna", "Annamaria","Antonia","Antonietta", "Antonina", "Antosia", "Anula", "Anusia", "Apolonia", "Arabela","Areta", "Ariadna", "Ariel", "Arlena", "Arleta", "Armida", "Ascelina", "Asia", "Asteria", "Atena", "Atina", "Augusta", "Augustyna", "Aurelia", "Aurora", "Aza", "Balbina", "Balladyna", "Barbara","Barbora", "Basia", "Basieńka", "Basiuchna", "Basiunia", "Baśka", "Bazylisa", "Bea", "Beata", "Beatka", "Beatrycze", "Benedykta", "Beniamina","Benigna", "Berbla", "Berenika", "Bernadeta", "Berta", "Bianka", "Bibiana", "Blandyna", "Blanka", "Błażeja", "Bogdana", "Bogna", "Boguchwała",
"Bogumiła", "Bogusława", "Bohdana", "Bolesława","Bona", "Bożena", "Bożenna", "Bronisława", "Brunhilda", "Brygida", "Carmen", "Cecylia","Celestyna", "Celia", "Celina", "Cezaria", "Chloe", "Ciechosława", "Czesia", "Cześka", "Czesława", "Dąbrówka", "Dafne", "Dagmara", "Dagna", "Dalila", "Dalisława", "Damroka", "Dana", "Daniela", "Danisława", "Danka", "Danuśka", "Danuta", "Daria", "Daromiła", "Dawida","Debora", "Delfina", "Diana", "Dioniza", "Dobrawa", "Dobromiła", "Dobrota", "Dominika", "Donalda", "Donna", "Dora", "Dorota", "Dorotka", "Doryda", "Dwojra", "Dżamila", "Dżesika", "Edeltruda", "Edwarda", "Edyta", "Ela", "Elcia", "Elefteria", "Eleonora", "Eleukadia", "Eleuteria", "Elfryda", "Eligia", "Eliza", "Elka", "Elmira", "Elunia", "Elusia", "Elwira","Elżbieta", "Elżbietka", "Elżunia", "Emerencjana", "Emilia", "Emma", "Erentruda", "Ermentruda","Erna", "Ernestyna", "Erwina", "Eryka", "Estera", "Eufemia", "Eufrozyna", "Eugenia",  "Eunika", "Eunomia", "Eustachia", "Euzebia", "Ewa", "Ewalda", "Ewarysta", "Ewelina","Fabiana", "Fatima", "Faustyna", "Felicja", "Felicyta", "Felikula", "Filomena", "Flawia", "Flora", "Florencja", "Florentyna", "Florentynka", "Fortunata", "Franciszka", "Frania",  "Fredegunda", "Fredeswinda", "Freja", "Frida", "Fryda", "Fryderyka",  "Gabriela", "Gaja", "Gemma", "Genowefa", "Georgina", "Geraldyna", "Gerda", "Gertruda", 
"Gilda", "Gorgonia", "Gracja", "Grażyna", "Greta", "Grzymisława", "Gusta", "Gustla", "Hagar", "Halina", "Halszka", "Hanna", "Hela", "Helcia", "Helena", "Helenka", "Helga", "Helunia", "Helusia", "Henia", "Henryczka", "Henryka", "Hermenegilda", "Herminia", "Hestia", "Hiacynta", "Hieronima", "Hilaria", "Hilda", "Hildegarda", "Hildzia", "Honorata", "Hortensja", "Ida", "Ifigenia", "Iga", "Ilona", "Inga", "Ingeborga", "Innocenta", "Ira", "Irena", "Irenka", "Irma", "Irmina", "Iryda", "Ismena", 
"Iwona", "Izabela", "Izaura", "Izolda", "Izyda", "Jaczemira", "Jadwiga", "Jadzia", "Jaga", "Jagata", "Jagienka", "Jagna", "Jagoda", "Jagula", "Jagusia", "Janeczka", "Janina", "Janinka", "Janka", "Jessica", "Jewka", "Joanna", "Jokasta", "Jolanta", "Jolenta", "Jowita", "Józefa", "Józefina", "Judyta", "Julia", "Julianna", "Julita", "Julitta", "Junia", "Justyna", "Kaja", "Kalina", "Kamila", "Kamilcia", "Kamilka", "Kandake", "Kandea", "Kandyda", "Karina", "Karolina", "Kasandra", "Kasia", "Kasieńka", "Kaśka", "Katarzyna", "Katarzynka", "Kazia", "Kazimiera", "Kazka", "Keiko", "Kinga", "Kira", "Klara", "Klarcia", "Klarysa", "Klaudia", "Klaudyna", "Klementyna", "Klementynka", "Kleopatra", "Klio", "Klotylda", "Kolumba", "Konkordia", "Konstancja", "Konstantyna", "Kordelia", "Kornelia", "Korona", "Krycha", "Krysia", "Krystyna", "Krystynka", "Krzesia", "Krzesisława", "Krzysztofa", "Ksenia", "Ksymena", "Kunegunda", "Kwiatosława", "Kwieta", "Kwietosława", "Kwiryna", "Larysa", "Laura", "Laurencja", "Lena", "Leokadia", "Leonia", "Leontyna", "Leopolda", "Liberia", "Libia", "Lidia", "Ligia", "Liliana", "Lilianna", "Lina", "Linda", "Litka", "Litosława", "Liwia", "Liza", "Lodzia", "Lolita", "Longina", "Lora", "Lourdes", "Łucja", "Lucyna", "Ludgarda", "Ludmiła", "Ludwika", "Luiza", "Luna", "Luśka", "Lutgarda", "Lyjna", "Macieja", "Madonna", 
"Magda", "Magdalena", "Magdalenka", "Maja", "Maksencja", "Małgorzata", "Malwina", "Mańcia", "Manuela", "Mara", "Marcela", "Marcelina", "Marcjanna", "Marcysia", "Marencja", "Maria", "Marianna", "Marietta", "Marika", "Marina", "Mariola", "Marion", "Marisol", "Marlena", "Marta", "Martyna", "Marya", "Maryja", "Maryla", "Maryna", "Marynia", "Marzanna", "Marzena", "Matylda", "Medarda", "Melania", "Melina", "Melita", "Mercedes", "Michalina", "Mieczysława", "Mila", "Miła", "Milena", "Miłość", "Miłosława", "Mira", "Miranda", "Mirela", "Miriam", "Mirona", "Mirosława", "Monia", "Moniczka", "Monika", "Monisia", "Mścisława", "Nadia", "Nadzieja", "Narcyza", "Nastka", "Natalia", "Natasza", "Neferetiti", "Nefertiti", "Nefretete", "Nela", "Nijoła", "Nikodema", "Nikola", "Nikoletta", "Nina", "Noemi", "Nora", "Norberta", "Norma", "Nula", "Nur", "Odeta", "Odyseja", "Ofelia", "Ofka", "Oktawia",  "Olga", "Olimpia", "Oliwia", "Oriana", "Otylia""Pamela", "Pamelka", "Pandora", "Patrycja", "Paula", "Pauletta", "Paulina", "Pelagia", "Perpetua", "Petronela", "Pia", "Poliksenia", "Polina", "Pola", "Polyksena", "Prakseda", "Przybygniewa", 
"Pulcheria", "Rachela", "Rafaela", "Rajnolda", "Ramona", "Regina", "Remigia", "Renata", "Roberta", "Róża", "Różyczka", "Rozalia", "Rozalinda", "Rozamunda", "Rozanna", "Rozina", "Rudolfa", "Rufina", "Sabina", "Salomea", "Samanta", "Sandra", "Sara", "Saturnina", "Scholastyka", "Selena", "Serafina", "Sibilla", "Sidonia", "Sława", "Sławomira", "Sobiesława", "Sofia", "Solange", "Sonia", "Stanisława", "Stella", "Stefania", "Stefka", "Sylwia", "Sybilla", "Symeona", "Szymona", "Tacjana", "Tamara", "Teodora", "Teodota", "Teodozja", "Teofila", "Terencja", "Teresa", "Tereska", "Tomisława", "Tomasza", "Tosia", "Trifena", "Tristana", "Tyberia", "Tyburcja", "Tyra", "Ula", "Urszula", "Urszulka", "Wanesa", "Wanda", "Waleria", "Walery", "Waltrauda", "Waltruda", "Wanda", "Wąsława", "Werena", "Weronika", "Wiara", "Więcława", "Więcesława", "Wiera", "Wiktoria", "Wiktorina", "Wilemina", "Wilga", "Wilhelmina", "Wirginia", "Władysława", "Włodzimiera", "Wojciecha", "Wolimira", "Wrociwoja", "Zacharia", "Zachariasza", "Zbysława", "Zdzisława", "Zenobia", "Zenona", "Zina", "Zinaida", "Zita", "Zofia", "Zuzanna", "Żaklina", "Żaneta", "Żanna", "Żenia", "Żywia","Zyta", "Żanetta", "Żelisława"
              ]

In [116]:
male_names =["Aaron", "Aaronek", "Abachum", "Abakuk", "Abdiasz", "Abdon", "Abdul", "Abel", "Abelard", "Abihu", "Abraham", "Abram", "Absalom", "Abundiusz", "Achilles", "Achim", "Adalbert", "Adam", "Adaś", "Adolf", "Adrian", "Afrykan","Agapit", "Agatangelus", "Agaton", "Agesilaos", "Aggeusz", "Agrykola", "Agryppa", "Ajschylos", "Aladyn", "Alan","Albercik", "Albert", "Albin", "Albrecht", "Alcybiades", "Alek", "Aleksander", "Aleksy", "Alfeusz", "Alfons","Alfred", "Ali", "Alkibiades", "Alojz", "Alojzy", "Alwar", "Alwarez", "Amadeusz", "Ambroży", "Amilkar", "Amon","Amos", "Amosek", "Anaklet", "Ananiasz", "Anastazy", "Anatol", "Andrea", "Andrzej", "Anicet", "Anioł", "Annasz","Antek", "Anteros", "Antioch", "Antoni", "Antonin", "Antoniusz", "Antoś", "Anzelm", "Apolinary", "Apollin", "Apollo",
  "Apoloniusz", "Ararat", "Archelaus", "Archimedes", "Argymir", "Ariel", "Arkadiusz", "Arkady", "Arnold", "Aron","Arsakes", "Arseniusz", "Artakserkses", "Artem", "Artur", "Arystokles", "Arystoteles", "Askaniusz", "Atanazy", "August","Augustyn", "Aurelian", "Aureliusz", "Awentyn", "Aza", "Azor", "Baldwin", "Baltazar", "Barabasz", "Barack", "Barnaba", "Bartek", "Bartłomiej", "Bartosz", "Bartuś", "Baruch", "Bastek", "Baszar", "Baszszar", "Bazyli", "Beda", "Benedykt", "Beniamin", "Beniaminek", "Benicjusz", "Benigny", "Benon", "Berek", "Bernard", "Bernardyn", "Biernat", "Blandyn", "Błażej", "Bogdan", "Bogdar", "Boguchwał", "Bogumił", "Bogumir", "Bogusław", "Bohdan", "Bolesław", "Bolko", "Bonawentura", "Bonifacy", "Booz", "Boromeusz", "Borys", "Borysek", "Bożydar", "Brajan", "Brian", "Bronek", "Bronisław", "Bruno", "Brunon","Brutus", "Bryan", "Bułat", "Cecylian", "Cecyliusz", "Celestyn", "Cezariusz", "Cezary", "Chamor", "Charles", "Cherubin", "Chlodwig", "Chrodegang", "Chromacjusz", "Chrystian", "Chryzant", "Chryzogon", "Chryzostom", "Chuniald", "Ciechosław","Cieszygor", "Cieszymir", "Cypisek", "Cyprian", "Cyrus", "Cyryl", "Czaruś", "Czesiek", "Czesław", "Częstoch", "Dąb", "Dacjusz","Dagobert", "Dalmacjusz", "Damazy", "Damian", "Daniel", "Dante", "Darek", "Dariusz", "Dawid", "Delfin", "Demetriusz", "Demokryt","Diego", "Diogenes", "Dionizy", "Ditmar", "Długomił", "Długosz", "Dobiegniew", "Dobromił", "Dobromir", "Dobrosław", "Dominik","Donald", "Donalduś", "Donat", "Dorian", "Dydak", "Dygda", "Dymitr", "Dyzma", "Dziećmiar", "Dzietrzych", "Ebenezer", "Edek","Edgar", "Edmund", "Edom", "Eduś", "Edward", "Edwin", "Edzik", "Edzio", "Edziu", "Efialtes", "Efraim", "Efrem", "Efron", "Egon","Ela", "Elefterios", "Eleuteriusz", "Eleutery", "Eliasz", "Eligiusz", "Elred", "Emanuel", "Emeryk", "Emil", "Emilian", "Emir","Emmanuel", "Eneasz", "Engelbercik", "Engelbert", "Enniusz", "Enoch", "Epifaniusz", "Epikur", "Epimach", "Erazm", "Ernest","Eros", "Erwin", "Eryk", "Eucheriusz", "Eufemiusz", "Eugeniusz", "Eustachiusz", "Eustachy", "Eustazy", "Eutyches", "Euzebiusz","Ewald", "Ewangelos", "Ewaryst", "Ezana", "Ezaw", "Ezdrasz", "Ezechiasz", "Ezechiel", "Fabian", "Fabrycy", "Faust", "Faustyn",
  "Felicjan", "Feliks", "Ferdynand", "Fibicjusz", "Fidel", "Fidelis", "Fileasz", "Filemon", "Filip", "Filon", "Flawiusz", "Florek","Florencjusz", "Florenty", "Florentyn", "Florian", "Florianek", "Floryn", "Focjusz", "Fortunat", "Fotis", "Francek", "Franciszek","Franek", "Frank", "Fryc", "Fryderyk", "Frydolin", "Frygidian", "Fulcjusz", "Fulgencjusz", "Gabriel", "Gajus", "Gajusz", "Gaweł","Gay", "Gedeon", "Gelazjusz", "Gelazy", "George", "Georges", "Gerald", "Gerard", "German", "Gerontiusz", "Gerwazy", "Gienek","Gilbert", "Gilead", "Gliceriusz", "Goar", "Godehard", "Godfryd", "Godprzyd", "Goeryk", "Gondulf", "Gonsalwy", "Gordian", "Gorgoniusz","Gosbert", "Gotard", "Gotszalk", "Gracjan", "Graham", "Grzegorz", "Gualfard", "Gustaw", "Gwidon", "Habakuk", "Hadrian", "Hamilkar", "Hamlet", "Hannibal", "Haralampiusz", "Harald", "Hazdrubal", "Hektor", "Helmut", "Helwidiusz", "Heniek", "Henio", "Heniu", "Heniuś","Henryczek", "Henryk", "Herakles", "Herakliusz", "Herbert", "Herkules", "Herman", "Hermes", "Herod", "Herodot", "Heron", "Herszel", "Hiacynt", "Hieronim", "Hilary", "Hiob", "Hipolit", "Homer", "Honorat", "Honoriusz", "Horacy", "Hubert", "Hugo", "Hugon", "Hygin","Ibrahim", "Idzi", "Ignacy", "Ignaś", "Igor", "Ildefons", "Innocenty", "Ireneusz", "Irydion", "Iwan", "Iwo", "Iwon", "Izaak","Izajasz", "Izak", "Izrael", "Izydor", "Jacek", "Jaczemir", "Jafet", "Jakub", "Jałbrzyk", "Jan", "Jan Kanty", "Janek", "Janisław","Jankiel", "Janko", "January", "Janusz", "Jarema", "Jaromir", "Jarosław", "Jasiek", "Jawor", "Jazon", "Jędruś", "Jędrzej", "Jeremi","Jeremiasz", "Jerzman", "Jeronim", "Jerzy", "Józef", "Józefat", "József", "Judasz", "Juliusz", "Jurek", "Jurok", "Jury", "Józek","Julian", "Julius", "Jumah", "Jurek", "Józek", "Juliusz", "Jowisz", "Józek", "Jozue", "Juda", "Judasz", "Julian", "Juliusz", "Juniper", "Jur", "Jurand", "Jurek", "Juruś", "Justus", "Justyn", "Justynian", "Kacper", "Kacperek", "Kajetan", "Kajfasz", "Kajtek", "Kajtuś", "Kajus", 
 "Kajusz", "Kalasanty", "Kalikst", "Kamil", "Kamilek", "Kandyd", "Kanty", "Karel", "Karim", "Karlik", "Karmel", "Karol", "Kasander", "Kasjan", "Kassander", "Kastor", "Kazek", "Kazik", "Kazimierz", "Kazio", "Kaziu", "Kefas", "Kiejstut", "Killion", "Klaudiusz", "Klemens", "Klementyn", "Kleofas", "Klet", "Kolumb", "Kolumban", "Konkordiusz", "Konon", "Konrad", "Konstancjusz", "Konstandinos", "Konstans", "Konstanty", "Konstantyn", "Kordian", "Kornel", "Korneliusz", "Korneliuszek", "Kosma", "Kostas", "Kresceniusz", "Kryspin", "Kryspus", "Krystian", "Krystyn", "Krzesimir", "Krzesisław", "Krzych", "Krzyś", "Krzysiek", "Krzysio", "Krzysiu", "Krzysztof", "Krzysztofor", "Ksawery", "Kserkses", "Kuba", "Kubuś", "Kurt", "Kwiatosław", "Kwietosław", "Kwintus", "Ładysław", "Lambert", "Lambro", "Lasota", "Laurencjusz", "Laurenty", "Łazarz", "Lech", "Leokadiusz", "Leon", "Leonard", "Leonid", "Leopold", "Leoś", "Lesław", "Lestek", "Leszek", "Lew", "Lewi", "Liberiusz", "Liboriusz", "Lidian", "Linus", "Litosław", "Liwiusz", "Lojz", "Lojzek", "Lojzik", "Longin", "Lorens", "Lotar", "Lubencjusz", "Lubomir", "Lucek", "Lucjan", "Łucjan", "Lucjusz", "Lucjuszek", "Lucyfer", "Ludwiczek", "Ludwik", "Luka", "Łukasz", "Lullus", "Lupus", "Lutacjusz", "Lutomysł", "Maciej", "Maciuś", "Magnus", "Mahdi", "Makary", "Maksencjusz", "Maksym", "Maksymian", "Maksymilian", "Maksymin", "Malachiasz", "Malcheusz", "Malchos", "Malchus", "Manfred", "Manuel", "Marcel", "Marceli", "Marcelin", "Marcin", "Marcjan", "Marcyś", "Mardocheusz", "Mareczek", "Marek", "Maria", "Marian", "Marianek", "Marion", "Mariusz", "Marsyliusz", "Maruś", "Marych", "Maryn", "Masseo", "Matatiasz", "Mateusz", "Matysek", "Maurycjusz", 
 "Maurycy", "Medard", "Melaniusz", "Melchior", "Melchizedek", "Merkury", "Metody", "Metrofan", "Michał", "Michałek", "Micheasz", "Mieczysław", "Mieszko", "Mietek", "Mika", "Mike", "Mikołaj", "Miłosław", "Miłosz", "Miluś", "Miron", "Mirosław", "Miruś", "Moab", "Modest", "Modoald", "Mohamed", "Mojmir", "Mojżesz", "Mordechaj", "Mowgli", "Mroczesław", "Mścisław", "Mustafa", "Nadab", "Nahum", "Napoleon", "Narcyz", "Natalis", "Natan", "Natanael", "Neczypor", "Neftali", "Nehemiasz", "Nepomucen", "Neron", "Nestor", "Nikifor", "Nikodem", "Nikodemek", "Nikoś", "Nikuś", "Nil", "Noe", "Norbercik", "Norbert", "Norbi", "Nur", "Odo", "Odon", "Odoryk", "Odys", "Odyseusz", "Oktawian", "Oktawiusz", "Olaf", "Olafek", "Olbracht", "Olbrycht", "Oleg", "Olek", "Olgierd", "Oliwer", "Oliwier", "Omar", "Ondrasz", "Ondraszek", "Onufry", "Optat", "Orcio", "Orest", "Orlando", "Oskar", "Oswald", "Otis", "Otmar", "Otto", "Otton", "Owidiusz", "Ozeasz", "Pachomiusz", "Pacyfik", "Pafnucy", "Pakosław", "Pankracy", "Pantaleon", "Panten", "Parsifal", "Parys", "Paschalis", "Paschazjusz", "Paschazy", "Patryk", "Paulin", "Paweł", "Pelagiusz", "Peregryn", "Petroniusz", "Philippe", "Pierre", "Pio", "Piotr", "Piotrek", "Piotruś", "Pius", "Platon", "Pliniusz", "Plutarch", "Polikarp", "Poncjan", "Poncjusz", "Popiel", "Priam", "Prokop", "Prot", "Protazy", "Przemek", "Przemysł", "Przemysław", "Ptolemeusz", "Pyjter", "Pyrrus", "Racibor", "Radomił", "Radosław", "Radzimir", "Rafał", "Rajmund", "Ramzan", "Randoald", "Razumnik", "Remigiusz", "Riad", "Richard", 
"Richariusz", "Rino", "Robercik", "Robert", "Roch", "Rodryg", "Roger", "Roland", "Roman", "Romuald", "Ronald", "Rudi", "Rudolf", "Rufus", "Rufusek", "Rusłan", "Rychu", "Rysiek", "Rysio", "Rysiu", "Ryszard", "Sadok", "Saladyn", "Salomon", "Salwator", "Samar", "Sambor", "Samson", "Samuel", "Saturian", "Saturnin", "Satyr", "Saul", "Sawa", "Ścibor", "Scypio", "Scypion", "Seba", "Sebastian", "Sebastianek", "Sekwan", "Sem", "Septymiusz", "Serafin", "Sergiusz", "Seweryn", "Siemomysł", "Siemowit", "Sigisbald", "Skałosz", "Sławomir", "Sławosz", "Sobiebor", "Sofoniasz", "Sofroniusz", "Sokrates", "Solon", "Soter", "Spirydion", "Stanisław", "Starwit", "Stasiu", "Stawros", "Stefan", "Stefcio", "Stefek", "Sulejman", "Światozar", "Świerad", "Świętopełk", "Sychem", "Sydoniusz", "Sykstus", "Sylas", "Sylwan", "Sylweriusz", "Sylwester", "Sylwiusz", "Symeon", "Symmachus", "Symplicjusz", "Syrach", "Syracydes", "Syriusz", "Syrycjusz", "Szarbel", "Szaweł", "Szczepan", "Szczęsny", "Szewach", "Szlomo", "Szymek", "Szymon", "Szymuś", "Tadek", "Tadeusz", "Tadeuszek", "Talus", "Tarsycjusz", "Tarsycjuszek", "Telesfor", "Temistokles", "Tennessee", "Teodor", "Teodoryk", "Teodozjusz", "Teofan", "Teofil", "Tercjusz", "Tertulian", "Tezeusz", "Tobiasz", "Tomasz", 
"Tomaszek", "Tomcio", "Tomeczek", "Tomek", "Tomir", "Tomisław", "Tomuś", "Tosiek", "Trajan", "Tristan", "Tyberiusz", "Tyburcjusz", "Tycio", "Tycjan", "Tymon", "Tymoteusz", "Tytus", "Ubald", "Udo", "Ulryk", "Urban", "Uriasz", "Uriel", "Urlyk", "Ursus", "Ursycyn", "Ursyn", "Wacio", "Wacław", "Wahid", "Walcerz", "Waldemar", "Walenty", "Walerian", "Waleriusz", "Walery", "Walter", "Warcisław", "Waryn", "Wawrzyn", "Wawrzyniec", "Wenanty", "Wergiliusz", "Wicio", "Wierzbięta", "Wierzyn", "Wiesław", "Wiktor", "Wiktoryn", "Wilhelm", "Wincenty", "Winicjusz", "Wirgiliusz", "Wit", "Witold", "Witosław", "Władek", "Właduś", "Władysław", "Władzio", "Władziu", "Włodzimierz", "Włodzisław", "Wodzisław", "Wojciech", "Wojsław", "Wojtek", "Wolfgang", "Wratysław", "Wrocisław", "Wszerad", "Wyszymir", "Yngve", "Ywald", "Zabulon", "Zachariasz", "Zachary", "Zacheusz", "Zawisza", "Zbigniew", "Zbyszek", "Zbyszko", "Zdobysław", "Zdzisław", "Zebrzyd", "Zebulon", "Zefir", "Zefiryn", "Żegota", "Zenek", "Zenobiusz", "Zenon", "Ziemowit", "Zozym", "Zygbert", "Zygfryd", "Zygmunt"
            ]

In [120]:
import pandas as pd
combined_data=pd.read_csv("bookBeat_data10_1.csv")

# Funkcja do przypisania płci
def assign_gender(lector):
    if pd.isnull(lector) or lector == '':
        return None
    # Sprawdzenie, czy w kolumnie Lector jest wyrażenie "zespół lektorów" lub więcej niż jeden lektor
    if 'zespół lektorów' in lector or ',' in lector:
        return 'G' #group 
    
    # Sprawdzenie, czy lektor jest na liście imion męskich lub żeńskich
    names = lector.split()  # Podziel na części, aby sprawdzić każde imię
    gender = None
    for name in names:
        if name in male_names:
            if gender == 'F':
                return 'U'  # Mieszanka imion męskich i żeńskich
            gender = 'M'
        elif name in female_names:
            if gender == 'M':
                return 'U'  # Mieszanka imion męskich i żeńskich
            gender = 'F'
        else:
            gender = 'Tom'  # Słowo nie pasuje do żadnej z list

    return gender if gender else 'U'  # Jeżeli brak wyniku, to "U"

# Dodanie nowej kolumny Gender
combined_data['Gender'] = combined_data['Lector'].apply(assign_gender)

# Wyświetlenie wyników
print(combined_data['Gender'])


                                                Title  \
0                               NIEZDOBYTA. TOM 1 i 2   
1                                  U nas w Auschwitzu   
2                                            Heydrich   
3                                Pudełko z marzeniami   
4                                 Ostatnie takie lato   
...                                               ...   
10030                                   Słuchając psa   
10031                                 Honor złodzieja   
10032  Casanova. Krótka historia słynnego uwodziciela   
10033                        Ucieczka Apsary. Część 1   
10034                      Tkaczka chmur. SŁUCHOWISKO   

                                        Author             Lector  Book_Score  \
0                              MELISSA DARWOOD        Magda Karel         4.7   
1                             Tadeusz Borowski  Marcin Popczyński         4.7   
2                              Giancarlo Villa    Tomasz Ignaczak       

In [122]:
# Zapisanie wyniku do pliku CSV
combined_data.to_csv("bookBeat_data11.csv", index=False)


In [127]:
combined_data=pd.read_csv("bookBeat_data11.csv")

# Przenosimy wartości z 'Volume' do 'Volume_Info', jeśli w 'Volume_Info' jest "Nie dotyczy"
# i w 'Volume' istnieje wartość
combined_data.loc[(combined_data['Volume_Info'] == 'Nie dotyczy') & (combined_data['Volume'].notnull()), 'Volume_Info'] = combined_data['Volume']
# Usuwamy wartość z 'Volume' po przepisaniu do 'Volume_Info'
combined_data.loc[(combined_data['Volume_Info'] == combined_data['Volume']) & (combined_data['Volume'].notnull()), 'Volume'] = np.nan
# Usuwamy kolumnę 'Volume', jeśli jest pusta
if combined_data['Volume'].isnull().all():
    combined_data.drop(columns=['Volume'], inplace=True)

# Sprawdzamy, czy kolumna 'Volume' została usunięta
#print(combined_data[['Volume', 'Volume_Info']].head())


In [129]:
# Zapisanie wyniku do pliku CSV
combined_data.to_csv("bookBeat_data12.csv", index=False)


In [136]:
combined_data=pd.read_csv("bookBeat_data12.csv")
combined_data.drop(columns=['Gender'], inplace=True)

In [139]:
# Funkcja do przypisania płci

def assign_gender(lector):
    if pd.isnull(lector) or lector == '':
        return None
    
    # Sprawdzenie, czy w kolumnie Lector jest wyrażenie "zespół lektorów" lub więcej niż jeden lektor
    if 'zespół lektorów' in lector or ',' in lector:
        return 'G'  # Zespół lektorów
    
    # Sprawdzenie, czy lektor jest na liście imion męskich lub żeńskich
    names = lector.split()  # Podziel na części, aby sprawdzić każde imię
    gender = None
    tom_candidate = True  # Flaga, która sprawdzi, czy całość to 'Tom'
    
    for name in names:
        if name in male_names:
            if gender == 'F':
                return 'U'  # Mieszanka imion męskich i żeńskich
            gender = 'M'
            tom_candidate = False  # Znaleziono imię męskie
        elif name in female_names:
            if gender == 'M':
                return 'U'  # Mieszanka imion męskich i żeńskich
            gender = 'F'
            tom_candidate = False  # Znaleziono imię żeńskie
        else:
            # Słowo nie pasuje do żadnej z list
            tom_candidate = tom_candidate and True
    
    # Jeżeli wszystkie słowa nie pasują do żadnej z list
    if tom_candidate:
        return 'Tom'  # Całość to "Tom"
    
    # W przypadku, gdy nie znaleziono płci
    return gender if gender else 'U'  # Jeżeli brak wyniku, to "U"

# Dodanie nowej kolumny Gender
combined_data['Gender'] = combined_data['Lector'].apply(assign_gender)



In [140]:
# Zapisanie wyniku do pliku CSV
combined_data.to_csv("bookBeat_data13.csv", index=False)

In [148]:
combined_data=pd.read_csv("bookBeat_data13_1.csv")

In [146]:
# Przenoszenie wartości z Lector do Volume_Info dla Gender == 'Tom'
mask_tom = combined_data['Gender'] == 'Tom'

# Przeniesienie danych z Lector do Volume_Info (tylko tam, gdzie Volume_Info jest puste lub "Nie dotyczy")
combined_data.loc[mask_tom, 'Volume_Info'] = combined_data.loc[mask_tom, 'Volume_Info'].replace('Nie dotyczy', pd.NA)
combined_data.loc[mask_tom, 'Volume_Info'] = combined_data.loc[mask_tom, 'Volume_Info'].fillna(combined_data.loc[mask_tom, 'Lector'])

# Wyczyszczenie kolumny Lector dla tych wierszy
combined_data.loc[mask_tom, 'Lector'] = pd.NA

# Wyświetlenie wyników
#print(combined_data[['Gender', 'Lector', 'Volume_Info']])


In [149]:
# Maska dla Gender == 'Tom'
mask_tom = combined_data['Gender'] == 'Tom'

# Przenoszenie wartości z Lector do Volume_Info
combined_data.loc[mask_tom, 'Volume_Info'] = combined_data.loc[mask_tom, 'Volume_Info'].replace('Nie dotyczy', pd.NA).fillna(combined_data.loc[mask_tom, 'Lector'])

# Wyczyszczenie kolumny Lector dla tych wierszy
combined_data.loc[mask_tom, 'Lector'] = pd.NA

# Wyświetlenie wyników dla kontroli
print(combined_data[['Gender', 'Lector', 'Volume_Info']])


      Gender             Lector                              Volume_Info
0          M    Robert Michalak                              Nie dotyczy
1          G    zespół lektorów                    Tom 2 - Projekt Riese
2          M       Filip Kosior                 Tom 8 - Trylogia demonów
3          M  Marcin Popczyński        Tom 1 - Cykl o Komisarzu Wrońskim
4          M    Wojciech Masiak                          Tom 3 - Hajmdal
...      ...                ...                                      ...
10030    NaN                NaN                              Nie dotyczy
10031    NaN                NaN                     Tom 1 - Robert Karcz
10032    NaN                NaN  Tom 6 - Prostytucja. Niezwykła historia
10033    NaN                NaN                              Nie dotyczy
10034    NaN                NaN                              Nie dotyczy

[10035 rows x 3 columns]


In [3]:

combined_data.to_csv("bookBeat_data14.csv", index=False)


NameError: name 'combined_data' is not defined

In [18]:
import pandas as pd
import numpy as np
combined_data=pd.read_csv("bookBeat_data14.csv")
#combined_data.info()


In [19]:
import pandas as pd

# Lista głównych kategorii
main_categories = [
    "kryminały", "powieści", "literatura faktu", "romans", "biografie",
    "fantastyka", "dla dzieci", "rozwój osobisty", "dla młodzieży",
    "rozrywka i humor", "klasyka", "sci-fi", "nowele, poezja i dramat", "inne"
]

# Zamiana kategorii na małe litery
main_categories = [cat.lower() for cat in main_categories]

# Mapowanie podkategorii do głównych kategorii
category_mapping = {
    "dla dzieci": [
        "dla dzieci w wieku 3+", "duchy i zjawiska paranormalne dla dzieci",
        "książki przygodowe dla dzieci", "literatura faktu dla dzieci",
        "dla dzieci w wieku 9+", "zabawa dla dzieci", "dla dzieci w wieku 6+",
        "książki o zwierzętach i naturze dla dzieci", "klasyka dla dzieci",
        "sport dla dzieci", "dla dzieci w wieku 0+",
        "książki o miłości i przyjaźni dla dzieci", "bajki i baśnie"],
    "dla młodzieży": [ "dla młodzieży w wieku 12+", "dla młodzieży w wieku 14+" ],
    "romans": ["romans sensacyjny", "romans historyczny", "pierwsza miłość","gorący romans"],
    "nowele, poezja i dramat": ["nowele", "dramat", "nowele i opowiadania", "poezja","poezja i dramat" ],
    "kryminały": [ "powieści kryminalne", "thrillery psychologiczne", "kryminały klasyczne","thrillery", "horrory", "true crime", "powieści detektywistyczne","kryminały dla młodzieży"],
    "biografie": [ "biografie religijne i filozoficzne", "biografie polityczne i społeczne","biografie monarchów", "autobiografie", "biografie artystów"],
    "powieści": ["powieści obyczajowe", "powieści współczesne", "powieści historyczne","powieści historyczne dla młodzieży", 'sagi rodzinne'],
    "fantastyka": [ "fantastyka romantyczna", "fantastyka i sci-fi dla młodzieży"],
    "rozrywka i humor": ["satyry i parodie", "powieści humorystyczne", "hobby i wypoczynek"],
    "literatura faktu": ["literatura wojenna", "historia", "polityka i wojna" ],
    "rozwój osobisty": ["językoznawstwo i języki obce", "rodzina i relacje", "mindfulness i duchowość", "ekonomia i biznes", "zdrowie",'psychologia'],
    "inne": [ "audiobook", "e-book", "społeczeństwo i kultura","przyroda i nauka", "komiks i powieść graficzna",'sport','podróże','religia i filozofia','jedzenie i napoje','literatura i sztuka','muzyka i filmy','prawo'
]
}

# Rozszerzenie głównych kategorii o ich podkategorie
for main_cat, sub_cats in category_mapping.items():
    main_categories.extend([sub_cat.lower() for sub_cat in sub_cats])

# Funkcja do przypisania głównej kategorii (pierwszej dopasowanej)
def classify_category(categories):
    if pd.isnull(categories):
        return "inne"  # Jeśli brak danych, przypisz "inne"
    split_categories = [cat.strip().lower() for cat in categories.split(",")]
    for cat in split_categories:
        if cat in main_categories:
            return cat  # Zwróć pierwszą dopasowaną kategorię
    return "inne"  # Jeśli brak dopasowania, przypisz "inne"

# Aktualizacja kolumny Matched_Categories
combined_data["Matched_Categories"] = combined_data["Category"].apply(classify_category)

# Sprawdzenie nieznanych kategorii
def check_unknown_categories(categories):
    split_categories = [cat.strip().lower() for cat in categories.split(",")]
    unknown = [cat for cat in split_categories if cat not in main_categories]
    return unknown

combined_data["Unknown_Categories"] = combined_data["Category"].apply(check_unknown_categories)

# Wyświetlenie unikalnych nieznanych kategorii
unknown_categories = set([cat for sublist in combined_data["Unknown_Categories"] for cat in sublist if cat])
print("Nieznane kategorie:", unknown_categories)

# Filtr danych, gdzie istnieją nieznane kategorie
rows_with_unknowns = combined_data[combined_data["Unknown_Categories"].apply(len) > 0]
print(rows_with_unknowns[["Category", "Unknown_Categories"]])


Nieznane kategorie: set()
Empty DataFrame
Columns: [Category, Unknown_Categories]
Index: []


In [20]:
combined_data.to_csv("bookBeat_data15.csv", index=False)


In [21]:
# Zmiana nazw kolumn
combined_data.rename(columns={"Category": "Original_Category", "Matched_Categories": "Category"}, inplace=True)

# Zamiana wartości "Tom" na NA w kolumnie Gender
combined_data['Gender'] = combined_data['Gender'].replace('Tom', pd.NA)



In [22]:
combined_data.to_csv("bookBeat_data16.csv", index=False)

In [23]:
import pandas as pd

# Wczytaj dane
combined_data = pd.read_csv("bookBeat_data16.csv")

# Podział danych
data_with_lector = combined_data[combined_data["Lector"].notna()]  # Dane z informacją o lektorze
data_without_lector = combined_data[combined_data["Lector"].isna()]  # Dane bez informacji o lektorze

# Sprawdzenie liczby obserwacji w obu zbiorach
print(f"Liczba obserwacji z lektorem: {len(data_with_lector)}")
print(f"Liczba obserwacji bez lektora: {len(data_without_lector)}")

# Zapisanie danych do plików CSV
data_with_lector.to_csv("BookBeat_full_lector.csv", index=False)
data_without_lector.to_csv("BookBeat_without_lector.csv", index=False)


Liczba obserwacji z lektorem: 8101
Liczba obserwacji bez lektora: 1934


In [26]:
combined_data = pd.read_csv("BookBeat_with_lector.csv")

In [27]:
# Usuwanie kolumny 'Column_Name'
combined_data = combined_data.drop(columns=['Unknown_Categories'])

# Zamiana 0 na NaN w kolumnach Book_Score i Lector_Score
combined_data[['Book_Score', 'Lector_Score']] = combined_data[['Book_Score', 'Lector_Score']].replace(0, np.nan)

# Zapisanie przetworzonych danych
combined_data.to_csv("BookBeat.csv", index=False)

In [29]:
# Zakładamy, że 'Publication_Date' jest już w formacie datetime
combined_data['Publication_Date'] = pd.to_datetime(combined_data['Publication_Date'], errors='coerce')

# Filtracja wierszy, gdzie rok nie jest 1900
combined_data = combined_data[combined_data['Publication_Date'].dt.year != 1900]

In [30]:
# Zapisanie przetworzonych danych
combined_data.to_csv("BookBeat1.csv", index=False)